**Annotation automatique de l'expertise avec GPT-4 :**

ce notebook produit une annotation automatique de l'expertise sur les données de notre corpus, avec gpt-4. Le prompt mêle des techniques de role-play learning et de chain-of-thought learning.

premières installation, notamment liées à open ai :

In [ ]:
!pip install openai

In [ ]:
import openai
import pandas as pd
from tqdm import tqdm
import re

On initialise la clef client, à trouver sur le site de open ai:

In [ ]:
client = openai.OpenAI(api_key="")

On définit le prompt, le modèle que l'on veut sélectionner, et la boucle qui va itérer sur les données. On s'assure avec la boucle finale de ne récupérer que le score donné par le LLM, et aucun commentaire parasite :

Attention : gpt-4 est payant, vous devez recharger vos crédits sur le site d'open AI pour pouvoir utiliser cette version du LLM :

In [ ]:
def gpt_expertise_score(text):
    prompt = f"""Tu es un assistant chargé d'évaluer automatiquement le niveau d'expertise d'un texte. Tu dois suivre une procédure logique en 4 étapes, SANS EXCEPTION, pour attribuer un score de 1, 2 ou 3.

Tu dois réfléchir étape par étape, en te posant les 5 questions suivantes dans cet ordre :

---

1. **Le texte contient-il un ou plusieurs chiffres précis ?**
(Exemples : "23 %", "1982", "4,5°C", "12 personnes", "3 millions", "sur 5 ans", etc.) Les numéros de téléphone, les indications sur la durée de vidéos, sont exclus.
→ Si OUI, alors Score = **3** → FIN
→ Sinon, continuer

2. **Le texte contient-il une référence à un ouvrage, article scientifique, article de journal ?**
(Exemples : "selon cet article du Monde", "selon l'ouvrage publié par Monsieur Dupont".
→ Si OUI, alors Score = **3** → FIN
→ Sinon, continuer

3. **Le texte mentionne-t-il un titre ou une autorité académique ?**
(Exemples : "professeur", "docteur", "chercheur", "publication scientifique", "thèse", "CNRS", "colloque"etc.)
→ Si OUI, alors Score = **3** → FIN
→ Sinon, continuer

4. **Le texte contient-il une démarche scientifique ou une structure logique explicite ?**
(Exemples : définition de concepts, formulation d'hypothèse, méthode expérimentale, logique rigoureuse)
→ Si OUI, alors Score = **3** → FIN
→ Sinon, continuer

5. **Le texte contient-il une réflexion sur les méthodes scientifiques ?**
(Exemples : définition de concepts, réflexions sur la valeur de la prédiction scientifique, réflexions sur les métriques scientifiques, citation de disciplines scientifiques -climatologie, océanographie, hydrologie-)
→ Si OUI, alors Score = **3** → FIN
→ Sinon, continuer

6. **Le texte contient-il des chiffres vagues ?**
(Exemples : "dans les années 60", "environ 20%")
→ Si OUI, alors Score = **2** → FIN
→ Sinon, continuer


8. **Le texte propose-t-il une réflexion sur des acteurs politiques, sociaux, scientifiques avec un vocabulaire précis  ?**
(exemple : citation d'organismes comme le GIEC, l'OMS, d'acteurs politiques comme des nations, des assocations. Un vocabulaire précis doit être présent : "écologisme", "souveraineté", "indépendance", "propagande")
→ Si OUI, alors Score = **2**

9. ** Le texte propose t'il des réflexions sur les médias ?**
(ex : critique d'une politique ou d'un climat médiatique en France)
→ Si OUI, alors Score = **2** → FIN
→ Sinon, continuer

10 ** Le texte propose t'il une réflexion ou critique sur le discours écologiste ?
→ Si OUI, alors Score = **2** → FIN
→ Sinon, continuer

→ Sinon (texte flou, émotionnel, subjectif), Score = **1**

---

Voici le texte à analyser :

\"\"\"{text}\"\"\"

**IMPORTANT :** Tu dois répondre exactement au format suivant :

```markdown
Étape 1 : [Oui ou Non, avec justification]
Étape 2 : [Oui ou Non, avec justification]
Étape 3 : [Oui ou Non, avec justification]
Étape 4 : [Oui ou Non, avec justification]
Étape 5 : [Oui ou Non, avec justification]
Étape 6 : [Oui ou Non, avec justification]
Étape 7 : [Oui ou Non, avec justification]
Étape 8 : [Oui ou Non, avec justification]
Étape 9 : [Oui ou Non, avec justification]
Étape 10 : [Oui ou Non, avec justification]

Score : [1, 2 ou 3]
"""
    try:
        response = client.chat.completions.create(
            model="gpt-4o",
            messages=[
                {"role": "system", "content": "Tu es un assistant strict, qui applique les règles à la lettre."},
                {"role": "user", "content": prompt}
            ],
            temperature=0
        )

        answer = response.choices[0].message.content.strip()
        print("Réponse :", answer)

        # Extraction du score entre 1 et 3
        match = re.search(r"Score\s*:\s*\*?\*?([1-3])\*?\*?", answer)
        return int(match.group(1)) if match else None

    except Exception as e:
        print(f"Erreur : {e}")
        return None


On charge les données :

In [ ]:
df = pd.read_csv("/content/annos_francesca.csv")

On applique la boucle sur nos données. Aussi, on ajoute le package tqdm, qui permet de voir la progression de la tâche (le temps de calcul peut atteindre plus de dix heures en fonction de la taille des données mises en entrée).

In [ ]:
from tqdm import tqdm
tqdm.pandas()

df["expertise_gpt4"] = df["content"].progress_apply(gpt_expertise_score)

  2%|▏         | 2/100 [00:04<03:27,  2.12s/it]

Réponse : ```markdown
Étape 1 : Non, le texte ne contient pas de chiffres précis.
Étape 2 : Non, le texte ne contient pas de référence à un ouvrage, article scientifique, ou article de journal.
Étape 3 : Oui, le texte mentionne des chercheurs et des disciplines académiques comme l'océanographie et l'hydrologie.
Étape 4 : Non, le texte ne présente pas de démarche scientifique ou de structure logique explicite.
Étape 5 : Oui, le texte contient une réflexion sur les méthodes scientifiques, mentionnant des disciplines comme l'océanographie et l'hydrologie.
Étape 6 : Non, le texte ne contient pas de chiffres vagues.
Étape 7 : Non, le texte ne propose pas de réflexion sur des acteurs politiques, sociaux, ou scientifiques avec un vocabulaire précis.
Étape 8 : Non, le texte ne propose pas de réflexion sur les médias.
Étape 9 : Non, le texte ne propose pas de réflexion ou critique sur le discours écologiste.

Score : 3
```


  3%|▎         | 3/100 [00:08<04:34,  2.83s/it]

Réponse : ```markdown
Étape 1 : Non, le texte ne contient aucun chiffre précis.
Étape 2 : Non, le texte ne contient aucune référence à un ouvrage, article scientifique ou article de journal.
Étape 3 : Non, le texte ne mentionne aucun titre ou autorité académique.
Étape 4 : Non, le texte ne contient pas de démarche scientifique ou de structure logique explicite.
Étape 5 : Non, le texte ne contient pas de réflexion sur les méthodes scientifiques.
Étape 6 : Non, le texte ne contient pas de chiffres vagues.
Étape 7 : Non, le texte ne propose pas de réflexion sur des acteurs politiques, sociaux, scientifiques avec un vocabulaire précis.
Étape 8 : Non, le texte ne propose pas de réflexion sur les médias.
Étape 9 : Non, le texte ne propose pas de réflexion ou critique sur le discours écologiste.

Score : 1
```


  4%|▍         | 4/100 [00:12<05:21,  3.35s/it]

...

Réponse : ```markdown
Étape 1 : Oui, avec justification. Le texte mentionne "de 1980 à aujourd'hui - on a plus de 40 ans", ce qui inclut des chiffres précis.
Étape 2 : Non, avec justification. Le texte ne contient pas de référence explicite à un ouvrage, article scientifique ou article de journal.
Étape 3 : Non, avec justification. Le texte ne mentionne pas de titre ou d'autorité académique.
Étape 4 : Non, avec justification. Le texte ne présente pas de démarche scientifique ou de structure logique explicite.
Étape 5 : Non, avec justification. Le texte ne contient pas de réflexion sur les méthodes scientifiques.
Étape 6 : Non, avec justification. Le texte ne contient pas de chiffres vagues.
Étape 7 : Non, avec justification. Le texte ne propose pas de réflexion sur des acteurs politiques, sociaux, scientifiques avec un vocabulaire précis.
Étape 8 : Non, avec justification. Le texte ne propose pas de réflexion sur les médias.
Étape 9 : Non, avec justification. Le texte ne propose pas de

100%|██████████| 100/100 [06:19<00:00,  3.85s/it]

Réponse : ```markdown
Étape 1 : Oui, le texte mentionne "1,2°C", ce qui est un chiffre précis.
Étape 2 : Non, il n'y a pas de référence à un ouvrage, article scientifique ou article de journal.
Étape 3 : Non, il n'y a pas de mention d'un titre ou d'une autorité académique.
Étape 4 : Non, il n'y a pas de démarche scientifique ou de structure logique explicite.
Étape 5 : Non, il n'y a pas de réflexion sur les méthodes scientifiques.
Étape 6 : Non, il n'y a pas de chiffres vagues.
Étape 7 : Non, il n'y a pas de réflexion sur des acteurs politiques, sociaux, scientifiques avec un vocabulaire précis.
Étape 8 : Non, il n'y a pas de réflexion sur les médias.
Étape 9 : Non, il n'y a pas de réflexion ou critique sur le discours écologiste.

Score : 3
```


100%|██████████| 100/100 [06:24<00:00,  3.84s/it]

Réponse : ```markdown
Étape 1 : Non, le texte ne contient pas de chiffres précis.
Étape 2 : Non, le texte ne contient pas de référence à un ouvrage, article scientifique, ou article de journal.
Étape 3 : Oui, le texte mentionne une autorité académique en parlant de "scientifiques" et du "GIEC".
Étape 4 : Non, le texte ne contient pas de démarche scientifique ou de structure logique explicite.
Étape 5 : Non, le texte ne contient pas de réflexion sur les méthodes scientifiques.
Étape 6 : Non, le texte ne contient pas de chiffres vagues.
Étape 7 : Non, le texte ne propose pas de réflexion sur des acteurs politiques, sociaux, scientifiques avec un vocabulaire précis.
Étape 8 : Non, le texte ne propose pas de réflexion sur les médias.
Étape 9 : Non, le texte ne propose pas de réflexion ou critique sur le discours écologiste.

Score : 3
```


On sauvegarde le fichier (note : ce code ne marche que dans google colab).

In [ ]:
# Sauvegarde
df.to_csv("annotation_expertise_gpt4_sans_regle_familier.csv", index=False)
